#### FISH Image Converter-local (version 1)

In [1]:
# Import packages
import os
import shutil
from smb.SMBConnection import SMBConnection
import tempfile
import re

import matplotlib.pyplot as plt
import numpy as np

import tifffile
import bigfish
import bigfish.stack as stack
import bigfish.plot as plot
import bigfish.multistack as multistack
import bigfish.detection as detection


#### Input Image info:

In [2]:
#specify the location of your data on the NAS
input_directory = '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1'
output_directory = '/Volumes/home/01_imaging/ERM-1_GFP/280829_erm-1_ERM-1'
folder_name = input_directory.split(os.path.sep)[-1]
output_directory = os.path.join(output_directory, folder_name, 'images')

#specify what you have in every channel
Cy5 = "set-3 mRNA" #(asymmetrical control)
mCherry = "erm-1 mRNA" #(query mRNA)
FITC = "membrane" #(ignore in this case) This is where protein markers usually go.
DAPI = "DAPI"


#### Input NAS credentials:

In [3]:
# NAS credentials and connection information
nas_host = '129.82.125.224' #nas_server_hostname_or_ip'
nas_user = 'nalyt' #your_username
nas_password = 'OA7LNjjfK@f05S' #your_password
nas_share = 'WormHole'

In [4]:
# Connect to the NAS
conn = SMBConnection(nas_user, nas_password, 'python-client', nas_host, use_ntlm_v2=True)
conn.connect(nas_host, 445)

# Check if connection is successfully established
smb_directory_path = os.path.relpath(input_directory, '/Volumes/WormHole')
shared_files = conn.listPath(nas_share, smb_directory_path) # move into folder with raw data
if shared_files:
    print("Connection is established.")
else:
    print("Connection is not established.")
    
# #print the filenames, save in a list 
# filenames = [shared_file.filename for shared_file in shared_files if shared_file.filename.endswith(("_R3D.dv","_R3D_REF.dv"))]
# filenames = sorted(filenames)
# filenames

Connection is established.


####  Code to read images directly from the NAS and save as a png

In [5]:
#List all the color images and brightfield images in separate lists
image_color_paths = []
bf_paths = []

for shared_file in shared_files:
    file_path = os.path.join(input_directory, shared_file.filename) 
    if shared_file.filename.endswith("_R3D_D3D.dv"):
        # If it does, append it to the image_colors_paths list
        image_color_paths.append(file_path)
        image_color_paths = sorted(image_color_paths)
    elif shared_file.filename.endswith("_R3D_REF.dv"):
        # If it ends with "_R3D_REF.dv", append it to the bf_paths list
        bf_paths.append(file_path)
        bf_paths = sorted(bf_paths)

#Read all the image_color stacks and extract the image identifiers to save in "subdirectories"
all_image_color_stacks = []
subdirectories = []
for image_color_path in image_color_paths:
    image_color_stack = stack.read_dv(image_color_path, sanity_check=False)
    all_image_color_stacks.append(image_color_stack)
    file_name = os.path.splitext(os.path.basename(image_color_path))[0]
    parts = file_name.split('_')
    numeric_part = None
    for i, part in enumerate(parts):
        if part == 'R3D':
            numeric_part = parts[i - 1]
            break
    if numeric_part is not None:
        subdirectories.append(numeric_part)

#Read all the bf stacks
all_bf_stacks = []
for bf_path in bf_paths:
    bf_stack = stack.read_dv(bf_path, sanity_check=False)
    all_bf_stacks.append(bf_stack)

# check your lists
print(f'image colors read: {len(all_image_color_stacks)} {image_color_paths}')
print(f'bf images read: {len(all_bf_stacks)} {bf_paths}')
print(subdirectories) #uncomment to check image identifiers

image colors read: 15 ['/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_01_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_02_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_03_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_04_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_05_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_06_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_07_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_08_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_09_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_erm-1_ERM-1/280829_erm-1_ERM-1_10_R3D_D3D.dv', '/Volumes/WormHole/Naly_2/04_ERM-1-GFP/280829_e

In [6]:
i

4

In [7]:
# Now iterate through all_image_color_stacks and extract every channel, then store it in all_color_channel_list 

all_color_channel_list = []
for i, stack in enumerate(all_image_color_stacks):
    image_colors = all_image_color_stacks[i]
    
    # Assuming image_colors is a 4D array where the first dimension represents different channels
    for channel_index in range(image_colors.shape[0]):
        current_image = image_colors[channel_index, :, :]
        all_color_channel_list.append(current_image)
        

        # Create a subdirectory to save each image as a PNG file
        output_subdirectory = os.path.join(output_directory, f"{subdirectories[i]}")
        os.makedirs(output_subdirectory, exist_ok=True)

        channel_name = None
        if channel_index == 0:
            channel_name = Cy5
        elif channel_index == 1:
            channel_name = mCherry
        elif channel_index == 2:
            channel_name = FITC
        elif channel_index == 3:
            channel_name = DAPI
 
        # Save each image as a PNG file
        plot_filename = os.path.join(output_subdirectory, f"{subdirectories[i]}_{channel_name}_deconvolved.png")
        plt.imsave(plot_filename, np.max(current_image, axis=0), cmap='gray')  # Adjust cmap if needed

# Print the directory where the images are stored
print(f"{len(output_subdirectories)} images saved in {output_directory}")


#You can use the images in the plot function
def plot(all_color_channel_list):
    # Create subplots for each image in the list
    num_images = len(all_color_channel_list)
    fig, ax = plt.subplots(1, num_images, figsize=(24, 5))

    for i, image in enumerate(all_color_channel_list):
        ax[i].imshow(image, cmap='gray')
        ax[i].set_title(f"Image {i}")
        ax[i].axis('off')

    plt.show()

NameError: name 'output_subdirectories' is not defined